In [1]:
#Importamos las librerías
import pandas as pd
import numpy as np
import matplotlib as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn import tree
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.model_selection import GridSearchCV

import warnings
warnings.filterwarnings('once')

Empezamos con los datos de ocasionales estandarizados y  codificados:

In [2]:
df=pd.read_csv('data/totales_estand_cod.csv',index_col=0)
df.head(3)

,registro,clima,temperatura,sens_termica,humedad,viento,total,fecha_nueva,festividad,festividad_1,mes_bueno,dia_semana_nuevo,no_laboral_nuevo,año_map,estacion_map
0,1,2,-0.828948,-0.678635,1.279463,-0.376545,985,2018-01-01,New Year's Day,1,1,0,0,0,0
1,2,2,-0.722312,-0.740034,0.507498,0.882110,801,2018-01-02,NaN,0,1,1,1,0,0
2,3,1,-1.642047,-1.746977,-1.317148,0.861132,1349,2018-01-03,NaN,0,1,2,1,0,0


In [3]:
df.columns

Index(['registro', 'clima', 'temperatura', 'sens_termica', 'humedad', 'viento',
       'total', 'fecha_nueva', 'festividad', 'festividad_1', 'mes_bueno',
       'dia_semana_nuevo', 'no_laboral_nuevo', 'año_map', 'estacion_map'],
      dtype='object')

In [4]:
df.drop(['registro','fecha_nueva','festividad'], axis=1,inplace=True)
df.columns

Index(['clima', 'temperatura', 'sens_termica', 'humedad', 'viento', 'total',
       'festividad_1', 'mes_bueno', 'dia_semana_nuevo', 'no_laboral_nuevo',
       'año_map', 'estacion_map'],
      dtype='object')

In [5]:
X=df.drop(['total'],axis=1)
y=df['total']

In [6]:
# Separamos en train y test
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# Creamos objeto del modelo
arbol=DecisionTreeRegressor(random_state=0)
#Ajustamos el modelo
arbol.fit(x_train,y_train)

DecisionTreeRegressor(random_state=0)

In [7]:
# Obtenemos la raíz cuadrada del número de variables predictoras de nuestro árbol
max_features = np.sqrt(len(x_train.columns))
max_features

3.3166247903554

In [8]:
# Obtenemos la profundidad del árbol representado
arbol.tree_.max_depth

26

In [9]:
# Predicción para los datos de entrenamiento
y_pred_train1 = arbol.predict(x_train)
# Predicción para los datos de test
y_pred_test1 = arbol.predict(x_test)

In [10]:
def obtener_metricas(y_test_m, y_train_m, y_predict_test_m, y_predict_train_m, modelo):
    # En un diccionario, calcular las métricas de train y test
    resultados_metricas = {
        'MAE': [mean_absolute_error(y_test_m, y_predict_test_m), mean_absolute_error(y_train_m, y_predict_train_m)],
        'MSE': [mean_squared_error(y_test_m, y_predict_test_m), mean_squared_error(y_train_m, y_predict_train_m)],
        'RMSE': [np.sqrt(mean_squared_error(y_test_m, y_predict_test_m)), np.sqrt(mean_squared_error(y_train_m, y_predict_train_m))],
        'R2':  [r2_score(y_test_m, y_predict_test_m), r2_score(y_train_m, y_predict_train_m)],
        "set": ["test", "train"], 
        "modelo": [modelo, modelo]}

    # Pasamos a dataframe el diccionario para su mejor visualizacion
    df_resultados_metricas = pd.DataFrame(resultados_metricas)

    return df_resultados_metricas

In [11]:
df_resultados1= obtener_metricas(y_test,y_train,y_pred_test1,y_pred_train1,'Decision Tree totales Codif y Estand')
df_resultados1

,MAE,MSE,RMSE,R2,set,modelo
0,556.136986,542894.780822,736.813939,0.841391,test,Decision Tree totales Codif y Estand
1,0.000000,0.000000,0.000000,1.000000,train,Decision Tree totales Codif y Estand


In [12]:
# Definir diccionario con los hiperparámetros a modificar
# min_samples_split: número mínimo de observaciones que debe tener un nodo para dividirse, por defecto [10, 50, 100]
# min_samples_leaf: número mínimo de observaciones que debe tener cada uno de los nodos hijos para que se produzca la division, por defecto [10, 50, 100]
param = {"max_depth": [2,4,6], "max_features": [1,2,3], "min_samples_split": [10, 150, 300], "min_samples_leaf": [10,150,300]}

In [13]:
# Ajustar el modelo con GridSearchCV
# cv = 10 --> número de veces que hacemos la validación
# verbose = - 1 --> para no printear ningún mensaje en la pantalla
grid = GridSearchCV(estimator=DecisionTreeRegressor(), param_grid=param, cv=10, verbose=-1,
                        return_train_score=True, scoring="neg_mean_squared_error")

In [14]:
# Ajustar el modelo
grid.fit(x_train, y_train)

GridSearchCV(cv=10, estimator=DecisionTreeRegressor(),
             param_grid={'max_depth': [2, 4, 6], 'max_features': [1, 2, 3],
                         'min_samples_leaf': [10, 150, 300],
                         'min_samples_split': [10, 150, 300]},
             return_train_score=True, scoring='neg_mean_squared_error',
             verbose=-1)

In [15]:
# El método nos dice cuál es el mejor modelo,
mejor_modelo = grid.best_estimator_
mejor_modelo

DecisionTreeRegressor(max_depth=6, max_features=3, min_samples_leaf=10,
                      min_samples_split=10)

In [16]:
y_pred_test_dt= mejor_modelo.predict(x_test)
y_pred_train_dt= mejor_modelo.predict(x_train)

In [17]:
df_resultados2= obtener_metricas(y_test,y_train,y_pred_test_dt,y_pred_train_dt,'Decision Tree totales Codif y Estand II')
df_resultados2

,MAE,MSE,RMSE,R2,set,modelo
0,663.530574,760203.738111,871.896633,0.777903,test,Decision Tree totales Codif y Estand II
1,604.751047,689091.035966,830.115074,0.819406,train,Decision Tree totales Codif y Estand II


In [18]:
df.head(2)

,clima,temperatura,sens_termica,humedad,viento,total,festividad_1,mes_bueno,dia_semana_nuevo,no_laboral_nuevo,año_map,estacion_map
0,2,-0.828948,-0.678635,1.279463,-0.376545,985,1,1,0,0,0,0
1,2,-0.722312,-0.740034,0.507498,0.882110,801,0,1,1,1,0,0


Ahora con los datos sin estandarizar:

In [19]:
df2 = pd.read_csv('data/totales_encod.csv', index_col = 0)

In [20]:
df2.columns

Index(['registro', 'clima', 'temperatura', 'sens_termica', 'humedad', 'viento',
       'total', 'fecha_nueva', 'festividad', 'festividad_1', 'mes_bueno',
       'dia_semana_nuevo', 'no_laboral_nuevo', 'año_map', 'estacion_map'],
      dtype='object')

In [21]:
df2.drop(['registro','fecha_nueva','festividad'], axis=1,inplace=True)
df2.columns

Index(['clima', 'temperatura', 'sens_termica', 'humedad', 'viento', 'total',
       'festividad_1', 'mes_bueno', 'dia_semana_nuevo', 'no_laboral_nuevo',
       'año_map', 'estacion_map'],
      dtype='object')

In [23]:
X2=df.drop(['total'],axis=1)
y2=df['total']

In [24]:
# Separamos en train y test
x_train2, x_test2, y_train2, y_test2 = train_test_split(X2, y2, test_size=0.2, random_state=42)
# Creamos objeto del modelo
arbol2=DecisionTreeRegressor(random_state=0)
#Ajustamos el modelo
arbol2.fit(x_train2,y_train2)

DecisionTreeRegressor(random_state=0)

In [25]:
# Obtenemos la raíz cuadrada del número de variables predictoras de nuestro árbol
max_features2 = np.sqrt(len(x_train2.columns))
max_features2

3.3166247903554

In [26]:
# Obtenemos la profundidad del árbol representado
arbol.tree_.max_depth

26

In [27]:
# Predicción para los datos de entrenamiento
y_pred_train2 = arbol2.predict(x_train2)
# Predicción para los datos de test
y_pred_test2 = arbol2.predict(x_test2)

In [28]:
df_resultados3= obtener_metricas(y_test2,y_train2,y_pred_test2,y_pred_train2,'Decision Tree totales Codif')
df_resultados3

,MAE,MSE,RMSE,R2,set,modelo
0,556.136986,542894.780822,736.813939,0.841391,test,Decision Tree totales Codif
1,0.000000,0.000000,0.000000,1.000000,train,Decision Tree totales Codif


In [29]:
# Definir diccionario con los hiperparámetros a modificar
# min_samples_split: número mínimo de observaciones que debe tener un nodo para dividirse, por defecto [10, 50, 100]
# min_samples_leaf: número mínimo de observaciones que debe tener cada uno de los nodos hijos para que se produzca la division, por defecto [10, 50, 100]
param2 = {"max_depth": [3,7,10], "max_features": [1,2,3], "min_samples_split": [10, 150, 300], "min_samples_leaf": [10,150,300]}

In [30]:
# Ajustar el modelo con GridSearchCV
# cv = 10 --> número de veces que hacemos la validación
# verbose = - 1 --> para no printear ningún mensaje en la pantalla
grid2 = GridSearchCV(estimator=DecisionTreeRegressor(), param_grid=param, cv=10, verbose=-1,
                        return_train_score=True, scoring="neg_mean_squared_error")

In [31]:
# Ajustar el modelo
grid2.fit(x_train2, y_train2)

GridSearchCV(cv=10, estimator=DecisionTreeRegressor(),
             param_grid={'max_depth': [2, 4, 6], 'max_features': [1, 2, 3],
                         'min_samples_leaf': [10, 150, 300],
                         'min_samples_split': [10, 150, 300]},
             return_train_score=True, scoring='neg_mean_squared_error',
             verbose=-1)

In [32]:
# El método nos dice cuál es el mejor modelo,
mejor_modelo2 = grid2.best_estimator_
mejor_modelo2

DecisionTreeRegressor(max_depth=4, max_features=3, min_samples_leaf=10,
                      min_samples_split=10)

In [33]:
y_pred_test_dt2= mejor_modelo2.predict(x_test2)
y_pred_train_dt2= mejor_modelo2.predict(x_train2)

In [36]:
df_resultados4= obtener_metricas(y_test2,y_train2,y_pred_test_dt2,y_pred_train_dt2,'Decision Tree totales Codif II')
df_resultados4

,MAE,MSE,RMSE,R2,set,modelo
0,708.723030,889245.178574,942.997974,0.740203,test,Decision Tree totales Codif II
1,699.336286,850216.969280,922.072106,0.777179,train,Decision Tree totales Codif II
